##GIT Repository

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
repo_url = 'https://github.com/gistairc/MUSIC4P3.git'
!git clone $repo_url

Cloning into 'MUSIC4P3'...
remote: Enumerating objects: 123, done.
remote: Total 123 (delta 0), reused 0 (delta 0), pack-reused 123
Receiving objects: 100% (123/123), 314.03 KiB | 3.27 MiB/s, done.
Resolving deltas: 100% (53/53), done.


#Data Process

##Imports

In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

In [4]:
folder_train = '/content/drive/MyDrive/TCC - Gabriela Alves Marinho/train'
folder_mask = '/content/drive/MyDrive/TCC - Gabriela Alves Marinho/masks'
folder_test = '/content/drive/MyDrive/TCC - Gabriela Alves Marinho/test'

train_files = os.listdir(folder_train)
mask_files = os.listdir(folder_mask)

train_files.sort()
mask_files.sort()

##Apply mask on train image to create the labels

In [5]:
labels = []

for mask_file in mask_files:
    mask_image = cv2.imread(os.path.join(folder_mask, mask_file))

    green = np.array([138, 164, 105])
    pink = np.array([190, 98, 130])
    tolerancy = 70

    # Verify if the image contains pink(erosion)
    is_erosion = np.any(np.all(np.abs(mask_image - pink) < tolerancy, axis=-1))


    # Add label
    labels.append(1 if is_erosion else 0)

###Separação da base de treino em treino e teste

In [6]:
# Combine as imagens e os rótulos em uma lista
combined_data = list(zip(train_files, labels))
train_data, test_data = train_test_split(combined_data, test_size=0.2, random_state=42)

# Separe os dados de treino e validação novamente
train_files, train_labels = zip(*train_data)
test_files, test_labels = zip(*test_data)

In [7]:
print(combined_data)
print(len(train_files))
print(len(test_files))

[('10.jpg', 0), ('100.jpg', 1), ('101.jpg', 1), ('102.jpg', 0), ('103.jpg', 1), ('104.jpg', 1), ('105.jpg', 1), ('106.jpg', 1), ('107.jpg', 1), ('108.jpg', 1), ('109.jpg', 1), ('11.jpg', 1), ('110.jpg', 1), ('111.jpg', 1), ('112.jpg', 1), ('113.jpg', 1), ('114.jpg', 1), ('115.jpg', 1), ('116.jpg', 1), ('117.jpg', 1), ('118.jpg', 1), ('119.jpg', 1), ('12.jpg', 1), ('120.jpg', 1), ('121.jpg', 1), ('122.jpg', 1), ('123.jpg', 1), ('124.jpg', 1), ('125.jpg', 1), ('126.jpg', 1), ('127.jpg', 1), ('128.jpg', 1), ('129.jpg', 1), ('13.jpg', 1), ('130.jpg', 0), ('131.jpg', 1), ('132.jpg', 1), ('133.jpg', 1), ('134.jpg', 1), ('135.jpg', 1), ('136.jpg', 1), ('137.jpg', 1), ('138.jpg', 1), ('139.jpg', 1), ('14.jpg', 1), ('140.jpg', 1), ('141.jpg', 1), ('142.jpg', 1), ('143.jpg', 1), ('144.jpg', 0), ('145.jpg', 1), ('146.jpg', 0), ('147.jpg', 1), ('148.jpg', 1), ('149.jpg', 1), ('15.jpg', 1), ('150.jpg', 1), ('151.jpg', 1), ('152.jpg', 1), ('153.jpg', 1), ('154.jpg', 1), ('155.jpg', 1), ('156.jpg', 1

In [ ]:
# Carregue os dados de treino e teste
train_data = []
for train_file, label in zip(train_files, train_labels):
    train_image = cv2.imread(os.path.join(folder_train, train_file))
    train_data.append((train_image, label))

test_data = []
for test_file in test_files:
    test_image = cv2.imread(os.path.join(folder_train, test_file))
    test_data.append(test_image)

#CNN Application

###Installs

In [ ]:
!pip install chainer

###Imports

In [ ]:
import numpy as np
import os

import chainer
from chainer import Function, gradient_check, Variable, optimizers, utils
from chainer import Link, Chain, ChainList
from chainer.training.extensions import Evaluator

import chainer.functions as F
import chainer.links as L
from chainer.datasets import LabeledImageDataset
from chainer import datasets

##Data

In [ ]:
class LabeledImageDataset(chainer.dataset.DatasetMixin):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def get_example(self, i):
        image, label = self.data[i]

        # Convert image data to float and normalize to the range [0, 1]
        image = image.astype(np.float32) / 255.0

        return image.transpose(2, 0, 1), label

In [ ]:
train_dataset = LabeledImageDataset(train_data)

###Class for CNN

In [ ]:
class DetectCNN(chainer.Chain):
    insize = 16
    def __init__(self):
      super(DetectCNN, self).__init__(
            conv1=L.Convolution2D(3, 32, 3, stride=1, pad=1),  # Adjusting padding to maintain spatial dimensions
            bn1=L.BatchNormalization(32),
            conv2=L.Convolution2D(32, 64, 3, stride=1, pad=1),  # Increasing the number of filters and adjusting padding
            bn2=L.BatchNormalization(64),
            conv3=L.Convolution2D(64, 128, 3, stride=1, pad=1),  # Increasing the number of filters and adjusting padding
            bn3=L.BatchNormalization(128),
            fcl=L.Linear(None, 2),  # Adjusting the size of the linear layer
        )
      self.train = True,

    def __call__(self, x, t=None):
            h = self.conv1(x)
            h = F.relu(h)
            h = F.max_pooling_2d(h, ksize=2, stride=2)  # Adding max pooling to reduce the spatial dimensions
            h = self.bn1(h)

            h = self.conv2(h)
            h = F.relu(h)
            h = F.max_pooling_2d(h, ksize=2, stride=2)  # Adding max pooling
            h = self.bn2(h)

            h = self.conv3(h)
            h = F.relu(h)
            h = F.max_pooling_2d(h, ksize=2, stride=2)  # Adding max pooling
            h = self.bn3(h)

            h = F.average_pooling_2d(h, ksize=h.shape[2:])

            h = self.fcl(h)
            h = F.dropout(h)

            if t is not None:
                # Durante o treinamento, calcule a entropia cruzada softmax
                self.pred = h
                self.loss = F.softmax_cross_entropy(h, t)
                self.accuracy = F.accuracy(self.pred, t)
                chainer.report({'loss': self.loss, 'accuracy': self.accuracy}, self)
                return self.loss
            else:
                # Durante a inferência, retorne as probabilidades diretamente
                self.pred = F.softmax(h)
                return self.pred

##Optimizers

In [ ]:
from chainer import optimizers

model = DetectCNN()
model.to_gpu()
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

##Iterator

In [ ]:
from chainer import iterators
from chainer.training import StandardUpdater, Trainer
from chainer.training.extensions import LogReport, PrintReport

batch_size = 32
train_iter = iterators.MultiprocessIterator(train_dataset, batch_size, shuffle=True)

##Training model

In [ ]:
updater = StandardUpdater(train_iter, optimizer, device=-1)
trainer = Trainer(updater, (60, 'epoch'), out='result')

# Log and print training progress
trainer.extend(LogReport())
trainer.extend(PrintReport(['epoch', 'main/loss', 'main/accuracy']))
trainer.run()

##Test

In [ ]:
class TestImageDataset(chainer.dataset.DatasetMixin):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def get_example(self, i):
        image = self.data[i]

        if image is None:
            print(f"Warning: Image at index {i} is None.")
            return None

        image = image.astype(np.float32) / 255.0
        return image.transpose(2, 0, 1)

In [ ]:
def predict_on_test(model, test_dataset):
    predictions = []
    with chainer.using_config('train', False):
        for i in range(len(test_dataset)):
            test_image = test_dataset[i]
            if test_image is not None:
                test_image = model.xp.asarray(test_image)
                test_image = test_image.reshape((1,) + test_image.shape)
                pred = model(test_image)
                if pred is not None:
                    probabilities = chainer.cuda.to_cpu(pred.array[0])
                    predictions.append(probabilities)
                else:
                    print(f"Warning: Skipping inference for None prediction at index {i}.")
            else:
                print(f"Warning: Skipping inference for None image at index {i}.")

    return predictions


In [ ]:
test_dataset = TestImageDataset(test_data)

model_test = DetectCNN()
model_test.train = False

test_predictions = predict_on_test(model_test, test_dataset)
print(test_predictions)

In [ ]:
from sklearn.metrics import accuracy_score

# Assuming you have ground truth labels for test data
true_labels = [label == 1 for label in test_labels]

# Convert probabilities to predicted labels
predicted_labels = [np.argmax(pred) for pred in test_predictions]

# Evaluate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy on test data: {accuracy}")

In [ ]:
for i, pred in enumerate(test_predictions):
    print(f"Example {i + 1} - Predicted Probabilities: {pred}")